In [1]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd

def load_mesh_from_folder(remeshed_path, clipped_path):
    """Load mesh data from a folder containing remeshed and clipped samples.
    Args:
        remeshed_path (str): Path to the folder containing remeshed samples.
        clipped_path (str): Path to the folder containing clipped samples.
    Returns:
        list: A list of tuples containing (remeshed_mesh_path, clipped_mesh_path).
    """
    mesh_pairs = []
    for samples_name in sorted(os.listdir(remeshed_path)):
        if not samples_name.startswith('sample_'):
            continue
        scalar_field_path = os.path.join(remeshed_path, samples_name, 'scalars.csv')
        remeshed_mesh_dir = os.path.join(remeshed_path, samples_name, 'meshes')
        clipped_mesh_dir = os.path.join(clipped_path, samples_name, 'meshes')
        
        if not os.path.isdir(remeshed_mesh_dir) or not os.path.isdir(clipped_mesh_dir):
            continue 
        if not os.path.isfile(scalar_field_path):
            print(f"⚠️ Scalar field file not found for {samples_name}. Skipping.")
            continue
            
        remeshed_mesh = os.path.join(remeshed_mesh_dir, 'mesh_000000000.cgns') 
        clipped_mesh = os.path.join(clipped_mesh_dir, 'mesh_000000000.cgns')
        scalar_df = pd.read_csv(scalar_field_path) 

        if os.path.isfile(remeshed_mesh) and os.path.isfile(clipped_mesh):
            mesh_pairs.append((remeshed_mesh, clipped_mesh, scalar_df))
            print(f"✅ Found mesh pair: {samples_name}") 
    
    return mesh_pairs

data_remeshed_path = './AirfRANS_remeshed/dataset/samples'
data_clipped_path = './AirfRANS_clipped/dataset/samples'

mesh_data = load_mesh_from_folder(data_clipped_path, data_remeshed_path) 
print(f"Found {len(mesh_data)} mesh pairs.")

✅ Found mesh pair: sample_000000000
✅ Found mesh pair: sample_000000001
✅ Found mesh pair: sample_000000002
✅ Found mesh pair: sample_000000003
✅ Found mesh pair: sample_000000004
✅ Found mesh pair: sample_000000005
✅ Found mesh pair: sample_000000006
✅ Found mesh pair: sample_000000007
✅ Found mesh pair: sample_000000008
✅ Found mesh pair: sample_000000009
✅ Found mesh pair: sample_000000010
✅ Found mesh pair: sample_000000011
✅ Found mesh pair: sample_000000012
✅ Found mesh pair: sample_000000013
✅ Found mesh pair: sample_000000014
✅ Found mesh pair: sample_000000015
✅ Found mesh pair: sample_000000016
✅ Found mesh pair: sample_000000017
✅ Found mesh pair: sample_000000018
✅ Found mesh pair: sample_000000019
✅ Found mesh pair: sample_000000020
✅ Found mesh pair: sample_000000021
✅ Found mesh pair: sample_000000022
✅ Found mesh pair: sample_000000023
✅ Found mesh pair: sample_000000024
✅ Found mesh pair: sample_000000025
✅ Found mesh pair: sample_000000026
✅ Found mesh pair: sample_00

In [2]:
def process_mesh_pair(remeshed_path, clipped_path ):
    """
    Process a pair of remeshed and clipped meshes with consistent scaling
    Args:
        remeshed_path (str): Path to remeshed mesh file
        clipped_path (str): Path to clipped mesh file   
    Returns:
        tuple: (remeshed_data, clipped_data, global_ranges) for consistent scaling
    """
    
    def extract_mesh_data(filepath, mesh_type):
        """Extract data from a single mesh file"""
        print(f"\nProcessing {mesh_type} mesh: {os.path.basename(filepath)}")
        
        # Read the mesh
        reader = pv.CGNSReader(filepath)
        reader.load_boundary_patch = False
        mesh = reader.read().combine()
        
        # Get bounds and create sampling plane
        bounds = mesh.bounds
        x_min, x_max, y_min, y_max, z_min, z_max = bounds
        resolution = 512
        
        plane = pv.Plane(center=[(x_min + x_max)/2, (y_min + y_max)/2, 0], 
                        direction=[0, 0, 1], 
                        i_size=x_max - x_min, 
                        j_size=y_max - y_min, 
                        i_resolution=resolution, 
                        j_resolution=resolution)
        
        # Sample the data
        sampled_mesh = plane.sample(mesh)
        n_points = sampled_mesh.n_points
        actual_resolution = int(np.sqrt(n_points))
        
        print(f"  Available arrays: {sampled_mesh.array_names}")
        print(f"  Resolution: {actual_resolution}x{actual_resolution}")
        
        # Extract data arrays
        data_dict = {}
        for array_name in ['nut', 'p', 'Ux', 'Uy', 'implicit_distance']:
            if array_name in sampled_mesh.array_names:
                data = sampled_mesh[array_name]
                
                # Reshape data
                if len(data) == actual_resolution * actual_resolution:
                    data = data.reshape(actual_resolution, actual_resolution)
                else:
                    # Interpolate to regular grid
                    from scipy.interpolate import griddata
                    points = sampled_mesh.points
                    x, y = points[:, 0], points[:, 1]
                    xi = np.linspace(x.min(), x.max(), resolution)
                    yi = np.linspace(y.min(), y.max(), resolution)
                    Xi, Yi = np.meshgrid(xi, yi)
                    data = griddata((x, y), data, (Xi, Yi), method='linear')
                
                data_dict[array_name] = data
                print(f"  {array_name}: range [{np.nanmin(data):.6f}, {np.nanmax(data):.6f}]")
        
        return data_dict, bounds
    
    # Process both meshes
    remeshed_data, remeshed_bounds = extract_mesh_data(remeshed_path, "REMESHED")
    clipped_data, clipped_bounds = extract_mesh_data(clipped_path, "CLIPPED")
    
    # Calculate global ranges for consistent scaling
    global_ranges = {}
    for var in ['nut', 'p', 'Ux', 'Uy']:
        if var in remeshed_data and var in clipped_data:
            # Find global min/max across both meshes
            remeshed_vals = remeshed_data[var]
            clipped_vals = clipped_data[var]
            
            global_min = min(np.nanmin(remeshed_vals), np.nanmin(clipped_vals))
            global_max = max(np.nanmax(remeshed_vals), np.nanmax(clipped_vals))
            
            global_ranges[var] = (global_min, global_max)
            print(f"\nGlobal range for {var}: [{global_min:.6f}, {global_max:.6f}]")
    
    return remeshed_data, clipped_data, global_ranges

In [3]:
def extract_airfoil_surface_and_compute_forces(data, mesh_type=""):
    """
    Extracts airfoil surface and computes pressure and viscous forces.
    This is a modular function to be called by the main coefficient calculation logic.
    """
    p = data.get('p')
    Ux = data.get('Ux')
    Uy = data.get('Uy')

    if p is None or Ux is None or Uy is None:
        print(f"⚠️  Warning: Missing one or more required fields ('p', 'Ux', 'Uy') in {mesh_type} data.")
        return 0, 0, 0, 0

    nut = data.get('nut', np.zeros_like(p))
    
    if 'implicit_distance' in data:
        distance = data['implicit_distance']
        surface_mask = np.abs(distance) < 0.01
    else:
        vel_mag = np.sqrt(Ux**2 + Uy**2)
        surface_mask = vel_mag < 0.1 * np.nanmax(vel_mag)
    
    if not np.any(surface_mask):
        print(f"⚠️  Warning: No airfoil surface found in {mesh_type} data.")
        return 0, 0, 0, 0
    
    surface_indices = np.where(surface_mask)
    n_surface_points = len(surface_indices[0])
    
    if n_surface_points < 10:
        print(f"⚠️  Warning: Very few surface points found ({n_surface_points}) in {mesh_type}.")
        return 0, 0, 0, 0
    
    print(f"✅ Found {n_surface_points} surface points in {mesh_type}")
    
    pressure_force_x, pressure_force_y = 0, 0
    viscous_force_x, viscous_force_y = 0, 0
    
    dy, dx = np.gradient(distance) if 'implicit_distance' in data else np.gradient(vel_mag)
    rho, mu = 1.0, 1.0
    total_surface_area = 0
    
    for i, j in zip(surface_indices[0], surface_indices[1]):
        if 'implicit_distance' in data:
            nx, ny = dx[i, j], dy[i, j]
            norm = np.sqrt(nx**2 + ny**2)
            if norm > 1e-12:
                nx /= norm
                ny /= norm
            else: continue
        else:
            nx, ny = 1.0, 0.0
        
        pressure_force_x += p[i, j] * nx
        pressure_force_y += p[i, j] * ny
        
        if i > 0 and i < p.shape[0]-1 and j > 0 and j < p.shape[1]-1:
            dudx = (Ux[i, j+1] - Ux[i, j-1]) / 2.0
            dudy = (Ux[i+1, j] - Ux[i-1, j]) / 2.0
            dvdx = (Uy[i, j+1] - Uy[i, j-1]) / 2.0
            dvdy = (Uy[i+1, j] - Uy[i-1, j]) / 2.0
            tau_xx = (mu + nut[i, j]) * 2 * dudx
            tau_yy = (mu + nut[i, j]) * 2 * dvdy
            tau_xy = (mu + nut[i, j]) * (dudy + dvdx)
            viscous_force_x += tau_xx * nx + tau_xy * ny
            viscous_force_y += tau_xy * nx + tau_yy * ny
        
        total_surface_area += 1
    
    if total_surface_area > 0:
        pressure_force_x /= total_surface_area
        pressure_force_y /= total_surface_area
        viscous_force_x /= total_surface_area
        viscous_force_y /= total_surface_area
    
    total_force_x = pressure_force_x + viscous_force_x
    total_force_y = pressure_force_y + viscous_force_y
    
    return total_force_x, total_force_y, pressure_force_x, viscous_force_x

def calculate_coefficients(data, mesh_type, angle_of_attack):
    """Helper to calculate coefficients for a single dataset."""
    print(f"\n--- Processing {mesh_type} Data ---")
    force_x, force_y, pressure_force, viscous_force = extract_airfoil_surface_and_compute_forces(data, mesh_type)
    
    alpha_rad = np.radians(angle_of_attack)
    cos_alpha, sin_alpha = np.cos(alpha_rad), np.sin(alpha_rad)
    
    drag_force = force_x * cos_alpha + force_y * sin_alpha
    lift_force = -force_x * sin_alpha + force_y * cos_alpha
    
    chord_length, rho_inf, V_inf = 1.0, 1.0, 1.0
    q_inf = 0.5 * rho_inf * V_inf**2
    
    drag_coefficient = drag_force / (q_inf * chord_length) if q_inf * chord_length != 0 else 0
    lift_coefficient = lift_force / (q_inf * chord_length) if q_inf * chord_length != 0 else 0
    pressure_drag_coeff = pressure_force / (q_inf * chord_length) if q_inf * chord_length != 0 else 0
    viscous_drag_coeff = viscous_force / (q_inf * chord_length) if q_inf * chord_length != 0 else 0
    
    print(f"Drag Force: {drag_force:.6f}, Lift Force: {lift_force:.6f}")
    
    return {
        "drag_coefficient": drag_coefficient,
        "lift_coefficient": lift_coefficient,
        "pressure_drag_coeff": pressure_drag_coeff,
        "viscous_drag_coeff": viscous_drag_coeff,
    }

def compute_drag_lift_coefficients_comparison(remeshed_data, clipped_data, global_ranges, angle_of_attack=0.0):
    """
    Computes and compares drag and lift coefficients from remeshed and clipped data.
    """
    remeshed_results = calculate_coefficients(remeshed_data, "REMESHED", angle_of_attack)
    clipped_results = calculate_coefficients(clipped_data, "CLIPPED", angle_of_attack)
    
    return remeshed_results, clipped_results

In [4]:
# --- Configuration ---
output_csv_path = 'coefficient_comparison_results.csv'
all_results = []

# --- Main Processing Loop ---
for remeshed_path, clipped_path, scalar_df in mesh_data:
    sample_name = os.path.basename(os.path.dirname(os.path.dirname(remeshed_path)))
    print(f"\n\n{'='*20} Processing Sample: {sample_name} {'='*20}")

    # Extract reference values from the scalar DataFrame
    if not scalar_df.empty:
        reference_conditions = {
            'C_D': scalar_df['C_D'].iloc[0],
            'C_L': scalar_df['C_L'].iloc[0],
            'angle_of_attack': scalar_df['angle_of_attack'].iloc[0],
            'inlet_velocity': scalar_df['inlet_velocity'].iloc[0]
        }
        angle_of_attack = reference_conditions['angle_of_attack']
    else:
        print(f"⚠️ Warning: scalar_df is empty for {sample_name}. Using default angle of attack.")
        angle_of_attack = 0.0
        reference_conditions = {}

    # Process the mesh pair to get field data
    remeshed_data, clipped_data, global_ranges = process_mesh_pair(remeshed_path, clipped_path)

    # Compute coefficients for both datasets
    remeshed_coeffs, clipped_coeffs = compute_drag_lift_coefficients_comparison(
        remeshed_data, clipped_data, global_ranges, angle_of_attack
    )

    # Helper for safe percentage difference calculation
    def calculate_diff(val1, val2):
        if abs(val1) > 1e-9:
            return 100 * (val2 - val1) / val1
        return float('inf') if val1 != val2 else 0.0

    # Store results for this sample, including all coefficients and their differences
    result_entry = {
        'sample': sample_name,
        
        'remeshed_cd': remeshed_coeffs['drag_coefficient'],
        'clipped_cd': clipped_coeffs['drag_coefficient'],
        'reference_cd': reference_conditions.get('C_D', 'N/A'),
        'cd_diff_percent': calculate_diff(remeshed_coeffs['drag_coefficient'], clipped_coeffs['drag_coefficient']),
        
        'remeshed_cl': remeshed_coeffs['lift_coefficient'],
        'clipped_cl': clipped_coeffs['lift_coefficient'],
        'reference_cl': reference_conditions.get('C_L', 'N/A'),
        'cl_diff_percent': calculate_diff(remeshed_coeffs['lift_coefficient'], clipped_coeffs['lift_coefficient']),

        'remeshed_pressure_drag': remeshed_coeffs['pressure_drag_coeff'],
        'clipped_pressure_drag': clipped_coeffs['pressure_drag_coeff'],
        'pressure_drag_diff_percent': calculate_diff(remeshed_coeffs['pressure_drag_coeff'], clipped_coeffs['pressure_drag_coeff']),

        'remeshed_viscous_drag': remeshed_coeffs['viscous_drag_coeff'],
        'clipped_viscous_drag': clipped_coeffs['viscous_drag_coeff'],
        'viscous_drag_diff_percent': calculate_diff(remeshed_coeffs['viscous_drag_coeff'], clipped_coeffs['viscous_drag_coeff']),
    }
    all_results.append(result_entry)

# --- Save Results to CSV ---
results_df = pd.DataFrame(all_results)
results_df.to_csv(output_csv_path, index=False)

print(f"\n\n✅ All samples processed. Results saved to '{output_csv_path}'")

# --- Display Final Comparison Table for the last sample ---
print("\n\n" + "="*60)
print(f"=== FINAL SAMPLE ({sample_name}) COMPARISON ===")
print("="*60)
print(f"| {'Metric':<25} | {'Remeshed':<12} | {'Clipped':<12} | {'Difference (%)':<15} |")
print(f"|{'-'*27}|{'-'*14}|{'-'*14}|{'-'*18}|")

for key in remeshed_coeffs:
    rem_val = remeshed_coeffs[key]
    clip_val = clipped_coeffs[key]
    
    if abs(rem_val) > 1e-9:
        diff = 100 * (clip_val - rem_val) / rem_val
    else:
        diff = float('inf') if clip_val != rem_val else 0.0
        
    print(f"| {key:<25} | {rem_val:<12.6f} | {clip_val:<12.6f} | {diff:<15.2f} |")

print("="*60)
display(results_df.head())



==================== Processing Sample: sample_000000000 ====================

Processing REMESHED mesh: mesh_000000000.cgns
  Available arrays: ['Ux', 'Uy', 'implicit_distance', 'nut', 'p', 'vtkOriginalPointIds', 'cell_ids', 'vtkOriginalCellIds', 'vtkValidPointMask', 'Normals', 'TextureCoordinates', 'vtkGhostType', 'vtkGhostType']
  Resolution: 513x513
  nut: range [0.000000, 0.005274]
  p: range [-580.084809, 474.401467]
  Ux: range [0.000000, 43.918259]
  Uy: range [-31.265326, 10.550596]
  implicit_distance: range [-3.354337, 0.000000]

Processing CLIPPED mesh: mesh_000000000.cgns
  Available arrays: ['Ux', 'Uy', 'implicit_distance', 'nut', 'p', 'zone_0', 'zone_1', 'zone_2', 'zone_3', 'zone_4', 'zone_5', 'vtkValidPointMask', 'Normals', 'TextureCoordinates', 'vtkGhostType', 'vtkGhostType']
  Resolution: 513x513
  nut: range [-0.000000, 0.005273]
  p: range [-580.088350, 474.201043]
  Ux: range [-0.000000, 43.906636]
  Uy: range [-31.224781, 10.547611]
  implicit_distance: range [-

,sample,remeshed_cd,clipped_cd,reference_cd,cd_diff_percent,remeshed_cl,clipped_cl,reference_cl,cl_diff_percent,remeshed_pressure_drag,clipped_pressure_drag,pressure_drag_diff_percent,remeshed_viscous_drag,clipped_viscous_drag,viscous_drag_diff_percent
0,sample_000000000,-31.404574,-31.200064,0.010637,-0.651213,-77.362372,-77.647355,-0.321047,0.368374,-5.966590,-5.883925,-1.385465,-25.535899,-25.414415,-0.475741
1,sample_000000001,-42.809603,-41.285300,0.010921,-3.560656,161.145464,153.908135,0.569220,-4.491177,-17.276218,-16.529881,-4.320030,-25.709486,-24.923611,-3.056749
2,sample_000000002,-151.851885,-151.361983,0.029255,-0.322618,463.658896,461.744132,1.620365,-0.412968,-133.939607,-133.426137,-0.383360,-19.847752,-19.863326,0.078468
3,sample_000000003,-34.697698,-34.791902,0.011212,0.271497,-81.797301,-82.292232,-0.309718,0.605070,-8.406206,-8.338329,-0.807459,-26.374747,-26.537330,0.616437
4,sample_000000004,-38.726895,-38.660600,0.011948,-0.171186,145.541946,145.064030,0.526712,-0.328370,-13.102824,-13.046711,-0.428251,-25.745977,-25.735395,-0.041103


In [9]:
# Advanced CFD-based drag and lift coefficient calculation

def advanced_force_calculation(mesh_data, reference_conditions):
    """
    Advanced method for computing drag and lift coefficients using CFD principles.
    
    This method implements:
    1. Proper surface integration using control volume approach
    2. Momentum equation-based force calculation
    3. Far-field force calculation for validation
    4. Proper handling of turbulent flows
    
    Args:
        mesh_data (dict): Flow field data
        reference_conditions (dict): Reference flow conditions
    
    Returns:
        dict: Comprehensive force analysis results
    """
    
    print("=== Advanced CFD Force Calculation ===")
    
    # Extract flow variables
    p = mesh_data['p']        # Pressure
    u = mesh_data['Ux']       # X-velocity
    v = mesh_data['Uy']       # Y-velocity
    nut = mesh_data.get('nut', np.zeros_like(p))  # Turbulent viscosity
    
    # Reference conditions
    rho_inf = reference_conditions.get('density', 1.225)      # kg/m³
    u_inf = reference_conditions.get('velocity', 50.0)       # m/s
    mu = reference_conditions.get('viscosity', 1.81e-5)      # Pa·s
    chord = reference_conditions.get('chord', 1.0)           # m
    
    # Method 1: Surface pressure integration (most common)
    def surface_pressure_integration():
        """Calculate forces by integrating pressure over airfoil surface"""
        
        # Find airfoil surface using level set or wall distance
        if 'implicit_distance' in mesh_data:
            surface_mask = np.abs(mesh_data['implicit_distance']) < 0.005
        else:
            # Alternative: use stagnation points and low velocity regions
            vel_mag = np.sqrt(u**2 + v**2)
            surface_mask = vel_mag < 0.05 * u_inf
        
        if not np.any(surface_mask):
            return 0, 0, "No surface found"
        
        # Get surface coordinates
        y_coords, x_coords = np.where(surface_mask)
        n_points = len(x_coords)
        
        if n_points < 20:
            return 0, 0, f"Insufficient surface points: {n_points}"
        
        # Sort points to form a closed contour
        # This is crucial for proper integration
        center_x, center_y = np.mean(x_coords), np.mean(y_coords)
        angles = np.arctan2(y_coords - center_y, x_coords - center_x)
        sorted_indices = np.argsort(angles)
        
        x_surface = x_coords[sorted_indices]
        y_surface = y_coords[sorted_indices]
        p_surface = p[y_surface, x_surface]
        
        # Calculate surface normals and tangents
        dx = np.diff(x_surface, append=x_surface[0])
        dy = np.diff(y_surface, append=y_surface[0])
        ds = np.sqrt(dx**2 + dy**2)  # Surface element length
        
        # Outward normal vectors (perpendicular to surface)
        nx = dy / (ds + 1e-12)  # Normal x-component
        ny = -dx / (ds + 1e-12) # Normal y-component
        
        # Integrate pressure forces
        # F = ∫ p * n * ds
        force_x = np.sum(p_surface * nx * ds)
        force_y = np.sum(p_surface * ny * ds)
        
        return force_x, force_y, f"Surface integration with {n_points} points"
    
    # Method 2: Control volume momentum analysis
    def control_volume_method():
        """Calculate forces using momentum equation in control volume"""
        
        # Define control volume boundaries (far from airfoil)
        h, w = p.shape
        margin = min(h, w) // 10
        
        # Inlet boundary (left)
        inlet_u = u[:, margin]
        inlet_p = p[:, margin]
        
        # Outlet boundary (right)  
        outlet_u = u[:, -margin]
        outlet_p = p[:, -margin]
        
        # Top and bottom boundaries
        top_v = v[margin, :]
        top_p = p[margin, :]
        bottom_v = v[-margin, :]
        bottom_p = p[-margin, :]
        
        # Momentum flux calculation
        # Force = ∫∫ (p + ρu²) dA - momentum flux
        
        # X-direction force (drag)
        inlet_momentum_flux = np.sum(rho_inf * inlet_u**2 + inlet_p)
        outlet_momentum_flux = np.sum(rho_inf * outlet_u**2 + outlet_p)
        
        drag_force = inlet_momentum_flux - outlet_momentum_flux
        
        # Y-direction force (lift)  
        top_momentum_flux = np.sum(rho_inf * top_v**2 + top_p)
        bottom_momentum_flux = np.sum(rho_inf * bottom_v**2 + bottom_p)
        
        lift_force = top_momentum_flux - bottom_momentum_flux
        
        return drag_force, lift_force, "Control volume momentum method"
    
    # Method 3: Viscous stress integration
    def viscous_stress_integration():
        """Add viscous stress contribution to forces"""
        
        # Calculate velocity gradients
        dudy, dudx = np.gradient(u)
        dvdy, dvdx = np.gradient(v)
        
        # Stress tensor components
        mu_total = mu + rho_inf * nut  # Total viscosity (laminar + turbulent)
        
        tau_xx = mu_total * 2 * dudx
        tau_yy = mu_total * 2 * dvdy
        tau_xy = mu_total * (dudy + dvdx)
        
        # For proper implementation, we need surface integration of viscous stress
        # This is a simplified version
        viscous_drag = np.sum(tau_xx) * (1.0 / (u.shape[0] * u.shape[1]))
        viscous_lift = np.sum(tau_xy) * (1.0 / (u.shape[0] * u.shape[1]))
        
        return viscous_drag, viscous_lift, "Viscous stress integration"
    
    # Calculate forces using multiple methods
    print("\nCalculating forces using multiple methods:")
    
    # Surface pressure integration
    fp_x, fp_y, method1_info = surface_pressure_integration()
    print(f"1. {method1_info}")
    print(f"   Pressure forces: Fx = {fp_x:.6f}, Fy = {fp_y:.6f}")
    
    # Control volume method
    fcv_x, fcv_y, method2_info = control_volume_method()
    print(f"2. {method2_info}")
    print(f"   CV forces: Fx = {fcv_x:.6f}, Fy = {fcv_y:.6f}")
    
    # Viscous stress
    fv_x, fv_y, method3_info = viscous_stress_integration()
    print(f"3. {method3_info}")
    print(f"   Viscous forces: Fx = {fv_x:.6f}, Fy = {fv_y:.6f}")
    
    # Combine results
    total_force_x = fp_x + fv_x
    total_force_y = fp_y + fv_y
    
    # Non-dimensionalize to get coefficients
    q_inf = 0.5 * rho_inf * u_inf**2  # Dynamic pressure
    
    cd = total_force_x / (q_inf * chord)
    cl = total_force_y / (q_inf * chord)
    
    # Also compute coefficients from control volume method for comparison
    cd_cv = fcv_x / (q_inf * chord)
    cl_cv = fcv_y / (q_inf * chord)
    
    results = {
        'cd_surface': cd,
        'cl_surface': cl,
        'cd_control_volume': cd_cv,
        'cl_control_volume': cl_cv,
        'pressure_forces': (fp_x, fp_y),
        'viscous_forces': (fv_x, fv_y),
        'reference_dynamic_pressure': q_inf,
        'chord_length': chord
    }
    
    print(f"\n=== FINAL RESULTS ===")
    print(f"Surface Integration Method:")
    print(f"  Cd = {cd:.6f}, Cl = {cl:.6f}")
    print(f"Control Volume Method:")
    print(f"  Cd = {cd_cv:.6f}, Cl = {cl_cv:.6f}")
    print(f"Reference dynamic pressure: {q_inf:.3f} Pa")
    
    return results

# Example usage with reference conditions
reference_conditions = {
    'density': 1.225,      # kg/m³ (air at sea level)
    'velocity': 50.0,      # m/s (freestream velocity)
    'viscosity': 1.81e-5,  # Pa·s (air viscosity)
    'chord': 1.0,          # m (airfoil chord length)
}

print("Demonstrating advanced force calculation method...")
print("Note: This would use the remeshed_data for analysis")
print("To run: advanced_results = advanced_force_calculation(remeshed_data, reference_conditions)")
advanced_results = advanced_force_calculation(remeshed_data, reference_conditions)
adv_results_clipped = advanced_force_calculation(clipped_data, reference_conditions)

Demonstrating advanced force calculation method...
Note: This would use the remeshed_data for analysis
To run: advanced_results = advanced_force_calculation(remeshed_data, reference_conditions)
=== Advanced CFD Force Calculation ===

Calculating forces using multiple methods:
1. Surface integration with 1569 points
   Pressure forces: Fx = 935238.120048, Fy = 868989.901346
2. Control volume momentum method
   CV forces: Fx = -17923.742819, Fy = 237268.923181
3. Viscous stress integration
   Viscous forces: Fx = 0.000050, Fy = 0.000084

=== FINAL RESULTS ===
Surface Integration Method:
  Cd = 610.767752, Cl = 567.503609
Control Volume Method:
  Cd = -11.705301, Cl = 154.951134
Reference dynamic pressure: 1531.250 Pa
=== Advanced CFD Force Calculation ===

Calculating forces using multiple methods:
1. Surface integration with 1592 points
   Pressure forces: Fx = 104671.585613, Fy = 1631087.544814
2. Control volume momentum method
   CV forces: Fx = -7743.435356, Fy = 237461.455755
3. Vis